We first need to do a summary of what we did in the last two notebooks. :-) Essentially we created a dataframe containing postal code, borough, neighborhoods, latitude and longitude and saved the data in a file, neighborhoods_with_location.csv.

First we need to import some libraries. Note that on my computer folium is already installed.

In [1]:
import numpy as np
import folium
import pandas as pd

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Let's first create a map of Toronto together with the neighborhoods. :)

In [8]:
df = pd.read_csv('neighborhoods_with_location.csv')
map_toronto = folium.Map(location = [43.761539, -79.411079], zoom_start = 11)
# add markers to map
for pc, lat, lng, borough, neighborhood in zip(df['PostalCode'], df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {} {}'.format(neighborhood, borough, pc)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

Awesome! Now we need to get our Foursquare credentials prepared!

In [9]:
LIMIT = 100 #We want at most 100 venues
CLIENT_ID = '35JVTJQ3DWXUWRZWIVRBAPEDD515TCLODKSVFL1TMKE0VOME' # your Foursquare ID
CLIENT_SECRET = 'UHOTU4NXUEBTO15T4BA3XN0032VU1TZ1OGSMXDRMCO5NGMMK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Now we need to import the function from the New York lab.

In [18]:
import requests # library to handle requests
#This is entirely from the New York lab
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

Now let's see what we have.

In [20]:
print(toronto_venues.shape)

(2243, 7)


In [21]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Now let's see how many venues are there near each neighborhood.

In [23]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Downsview North, Wilson Heights",16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,54,54,54,54,54,54


It seems that there are some neighborhoods with no or very few venues nearby. Well, that's fine. Now let's find out how many venue categories exist.

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Time to convert 'Venue Category' into dummy variables. **But before that..we can see that "Neighborhood" is actually a venue! Hence we have to use a different column name, 'Neighborhoods' for our original neighborhoods.**

In [47]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [52]:
toronto_onehot.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's see the shape of our dataframe.

In [53]:
toronto_onehot.shape

(2243, 277)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [55]:
toronto_mean = toronto_onehot.groupby('Neighborhoods').mean().reset_index()
toronto_mean.head()

,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


Now it's time for us to get the 10 most common types of venues near a neighborhood. In order to do so let's import more functions from the New York lab. **Note that when there exists less than 10 types of venues near a neighborhood then the rest of the venues will simply be picked randomly, hence we will exclide such neighborhoods.**

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    if row_categories_sorted.iloc[num_top_venues - 1] < 0.005:# equivalent to = 0 for doubles in our context
        return False, []
    return True, row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_mean['Neighborhoods']

for ind in np.arange(toronto_mean.shape[0]):
    test, data = return_most_common_venues(toronto_mean.iloc[ind, :], num_top_venues)
    if test:
        neighborhoods_venues_sorted.iloc[ind, 1:] = data

In [64]:
neighborhoods_venues_sorted

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Restaurant,Gym,Hotel,Bar,Clothing Store
1,Agincourt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Agincourt North, L'Amoreaux East, Milliken, St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Bank,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Coffee Shop
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Fried Chicken Joint,Pizza Place,Shopping Mall,Video Store,Fast Food Restaurant,Bridal Shop
6,Bayview Village,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Hardware Store,Pharmacy,Butcher,Restaurant,Café,Pub
8,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Pub
9,"Birch Cliff, Cliffside West",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ah..NaNs. We have to get rid of them.

In [72]:
neighborhoods_venues_cleaned = neighborhoods_venues_sorted.dropna().reset_index(drop = True)

In [110]:
neighborhoods_venues_cleaned

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Restaurant,Gym,Hotel,Bar,Clothing Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Bank,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Coffee Shop
2,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Fried Chicken Joint,Pizza Place,Shopping Mall,Video Store,Fast Food Restaurant,Bridal Shop
3,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Hardware Store,Pharmacy,Butcher,Restaurant,Café,Pub
4,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Pub
5,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Furniture / Home Store,Pet Store,Convenience Store,Climbing Gym,Caribbean Restaurant,Restaurant
6,Business reply mail Processing Centre969 Eastern,Yoga Studio,Garden Center,Smoke Shop,Skate Park,Spa,Farmers Market,Fast Food Restaurant,Brewery,Light Rail Station,Burrito Place
7,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Harbor / Marina,Airport Gate,Airport,Boat or Ferry
8,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Market,Bakery,Park,Italian Restaurant,Indian Restaurant,Chinese Restaurant,Pub
9,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Ice Cream Shop,Chinese Restaurant,Thai Restaurant


Now we have 44 neighborhoods left. What can we say about them?

Time for clustering. :-)

In [112]:
neighborhoods_venues_cleaned[['Neighborhoods']]

,Neighborhoods
0,"Adelaide, King, Richmond"
1,"Alderwood, Long Branch"
2,"Bathurst Manor, Downsview North, Wilson Heights"
3,"Bedford Park, Lawrence Manor East"
4,Berczy Park
5,"Brockton, Exhibition Place, Parkdale Village"
6,Business reply mail Processing Centre969 Eastern
7,"CN Tower, Bathurst Quay, Island airport, Harbo..."
8,"Cabbagetown, St. James Town"
9,Central Bay Street


toronto_mean needs to be reduced to 44 neighborhoods before we can use k-means to do the clustering.

In [118]:
toronto_mean_cleaned = toronto_mean.merge(right = neighborhoods_venues_cleaned[['Neighborhoods']], left_on = 'Neighborhoods', right_on = 'Neighborhoods')
toronto_clustering = toronto_mean_cleaned.drop('Neighborhoods', axis = 1)
#toronto_clustering

In [120]:
kclusters = 5
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)
kmeans.labels_

array([0, 3, 3, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 0, 3, 3, 0, 3, 1, 0, 1, 0,
       0, 0, 3, 3, 1, 4, 1, 0, 1, 0, 0, 3, 0, 0, 0, 0, 3, 3, 3, 1, 0, 2],
      dtype=int32)

Now it is time to put these neighborhoods on a map. So let's adapt more code from the New York lab.

In [121]:
kmeans.labels_.shape
#neighborhoods_venues_cleaned.insert(loc = 0, column = 'Cluster Label', value = kmeans.labels_)

(44,)

As usual we have to reduce everything to 44 neighborhoods.

In [122]:
neighborhoods_venues_cleaned.insert(loc = 0, column = 'Cluster Label', value = kmeans.labels_)
neighborhoods_venues_cleaned

,Cluster Label,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Restaurant,Gym,Hotel,Bar,Clothing Store
1,3,"Alderwood, Long Branch",Pizza Place,Gym,Pool,Skating Rink,Bank,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Coffee Shop
2,3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Fried Chicken Joint,Pizza Place,Shopping Mall,Video Store,Fast Food Restaurant,Bridal Shop
3,3,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Hardware Store,Pharmacy,Butcher,Restaurant,Café,Pub
4,0,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Pub
5,0,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Furniture / Home Store,Pet Store,Convenience Store,Climbing Gym,Caribbean Restaurant,Restaurant
6,3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Garden Center,Smoke Shop,Skate Park,Spa,Farmers Market,Fast Food Restaurant,Brewery,Light Rail Station,Burrito Place
7,3,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Harbor / Marina,Airport Gate,Airport,Boat or Ferry
8,0,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Market,Bakery,Park,Italian Restaurant,Indian Restaurant,Chinese Restaurant,Pub
9,0,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Ice Cream Shop,Chinese Restaurant,Thai Restaurant


In [124]:

toronto_merged = df.merge(right = neighborhoods_venues_cleaned, left_on = 'Neighborhood', right_on = 'Neighborhoods')
toronto_merged.drop(['Unnamed: 0', 'Neighborhoods'], axis = 1, inplace = True)
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Café,Historic Site,Beer Store
1,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Gym,Japanese Restaurant,Diner,Sushi Restaurant,Yoga Studio,Chinese Restaurant,Spa,Smoothie Shop,Seafood Restaurant
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,2,Fast Food Restaurant,Pizza Place,Athletics & Sports,Café,Bank,Intersection,Gastropub,Rock Climbing Spot,Gym / Fitness Center,Pharmacy
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Bubble Tea Shop,Thai Restaurant,Theater
4,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1,Coffee Shop,Asian Restaurant,Beer Store,Gym,Restaurant,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Italian Restaurant,Bike Shop
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Gastropub,Cocktail Bar,Cosmetics Shop,Bakery,Clothing Store,Park
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Pub
7,M4G,East York,Leaside,43.709060,-79.363452,1,Sporting Goods Shop,Coffee Shop,Grocery Store,Burger Joint,Supermarket,Beer Store,Bike Shop,Smoothie Shop,Shopping Mall,Breakfast Spot
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Ice Cream Shop,Chinese Restaurant,Thai Restaurant
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Coffee Shop,Diner,Convenience Store,Baby Store,Nightclub


In [126]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.761539, -79.411079], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Finally let's examine what the characteristics of each cluster is! :)

### Cluster 1

In [128]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Mexican Restaurant,Café,Historic Site,Beer Store
1,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Gym,Japanese Restaurant,Diner,Sushi Restaurant,Yoga Studio,Chinese Restaurant,Spa,Smoothie Shop,Seafood Restaurant
3,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,American Restaurant,Bubble Tea Shop,Thai Restaurant,Theater
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Gastropub,Cocktail Bar,Cosmetics Shop,Bakery,Clothing Store,Park
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Steakhouse,Farmers Market,Pub
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Ice Cream Shop,Chinese Restaurant,Thai Restaurant
12,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,0,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Restaurant,Gym,Hotel,Bar,Clothing Store
15,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Pizza Place,History Museum
16,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Café,Restaurant,Coffee Shop,Boutique,Cocktail Bar,New American Restaurant,Men's Store,Bakery,Vietnamese Restaurant
18,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Italian Restaurant,Gym,Gastropub,Deli / Bodega,Concert Hall


Hmm..Lots of coffee shops! That's a large cluster!

### Cluster 2

In [129]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1,Coffee Shop,Asian Restaurant,Beer Store,Gym,Restaurant,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Italian Restaurant,Bike Shop
7,M4G,East York,Leaside,43.709060,-79.363452,1,Sporting Goods Shop,Coffee Shop,Grocery Store,Burger Joint,Supermarket,Beer Store,Bike Shop,Smoothie Shop,Shopping Mall,Breakfast Spot
11,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Indian Restaurant,Sandwich Place,Yoga Studio,Bank,Intersection,Liquor Store,Discount Store,Park,Coffee Shop,Grocery Store
14,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Bus Station,Asian Restaurant,Restaurant,Shopping Mall,Electronics Store
26,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Chinese Restaurant,Rental Car Location,Salon / Barbershop,Sandwich Place
43,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,1,Gym,Thrift / Vintage Store,Convenience Store,Burger Joint,Sandwich Place,Discount Store,Social Club,Flower Shop,Bakery,Supplement Shop


### Cluster 3

In [130]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937,2,Fast Food Restaurant,Pizza Place,Athletics & Sports,Café,Bank,Intersection,Gastropub,Rock Climbing Spot,Gym / Fitness Center,Pharmacy


### Cluster 4

In [131]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Coffee Shop,Diner,Convenience Store,Baby Store,Nightclub
10,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,3,Coffee Shop,Sushi Restaurant,Restaurant,Pharmacy,Fried Chicken Joint,Pizza Place,Shopping Mall,Video Store,Fast Food Restaurant,Bridal Shop
13,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,3,Supermarket,Pharmacy,Bakery,Gym / Fitness Center,Park,Liquor Store,Brewery,Café,Bank,Discount Store
17,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Spa,Brewery,Bubble Tea Shop,Restaurant,Café
20,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Park,Sandwich Place,Liquor Store,Food & Drink Shop,Italian Restaurant,Sushi Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Brewery
23,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,3,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Coffee Shop,Hardware Store,Pharmacy,Butcher,Restaurant,Café,Pub
25,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,3,Furniture / Home Store,Café,Mexican Restaurant,Bar,Flea Market,Arts & Crafts Store,Fried Chicken Joint,Italian Restaurant,Bakery,Grocery Store
27,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Coffee Shop,Café,Sandwich Place,Pizza Place,Pharmacy,Indian Restaurant,Cosmetics Shop,Pub,Burger Joint,Liquor Store
29,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Dessert Shop,Sandwich Place,Pizza Place,Restaurant,Café,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Coffee Shop,Seafood Restaurant
31,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,3,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Italian Restaurant,Spa,Latin American Restaurant,Restaurant,Indie Movie Theater,Food & Drink Shop


### Cluster 5

In [132]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,M1W,Scarborough,"L'Amoreaux West, Steeles West",43.799525,-79.318389,4,Coffee Shop,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Discount Store,Grocery Store,Pharmacy,Sandwich Place,Pizza Place,Breakfast Spot
